Downloading the brain tumor MRI dataset from kaggle

In [ ]:
import kagglehub
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
print("path to dataset ",path)


path to dataset  /root/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1


Install codecarbon and efficientnet

In [ ]:
!pip install torch torchvision codecarbon efficientnet_pytorch


Prepare the data for training

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_path = path+"/Training"
test_path = path+"/Testing"

img_size = (256, 256)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(root=train_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Classes: ", train_dataset.classes)

Classes:  ['glioma', 'meningioma', 'notumor', 'pituitary']


Loading resnet18 and setting the loss function and optimizer

In [ ]:
from torchvision import models
from codecarbon import EmissionsTracker
import time
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)

model.fc = nn.Linear(model.fc.in_features, 4)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Training and emission tracking

In [ ]:
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

tracker = EmissionsTracker()

print("Starting energy tracking and model training...")

tracker.start()
start_time = time.time()

train_model(model, train_loader, criterion, optimizer, epochs=10)
evaluate_model(model, test_loader)

end_time = time.time()
tracker.stop()

print(f"Total Training Time: {end_time - start_time:.2f} seconds")

[codecarbon INFO @ 18:45:35] [setup] RAM Tracking...
[codecarbon INFO @ 18:45:35] [setup] CPU Tracking...
[codecarbon WARNING @ 18:45:35] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 18:45:37] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 18:45:37] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 18:45:37] [setup] GPU Tracking...
[codecarbon INFO @ 18:45:37] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:45:37] >>> Tracker's metadata:
[codecarbon INFO @ 18:45:37]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 18:45:37]   Python version: 3.11.11
[codecarbon INFO @ 18:45:37]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 18:45:37]   Available RAM : 12.675 GB
[codecarbon INFO @ 18:45:37]   CPU count: 2
[codeca

Starting energy tracking and model training...


[codecarbon INFO @ 18:45:52] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:45:52] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:45:52] Energy consumed for all GPUs : 0.000214 kWh. Total GPU Power : 51.27722808402282 W
[codecarbon INFO @ 18:45:52] 0.000411 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:46:07] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:46:07] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:46:07] Energy consumed for all GPUs : 0.000429 kWh. Total GPU Power : 51.606758710253025 W
[codecarbon INFO @ 18:46:07] 0.000822 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:46:22] Energy consumed for RAM : 0.000059 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:46:22] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codec

Epoch 1/10, Loss: 0.3086, Accuracy: 89.04%


[codecarbon INFO @ 18:46:37] Energy consumed for RAM : 0.000079 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:46:37] Energy consumed for all CPUs : 0.000708 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:46:37] Energy consumed for all GPUs : 0.000877 kWh. Total GPU Power : 53.72788788335525 W
[codecarbon INFO @ 18:46:37] 0.001664 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:46:52] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:46:52] Energy consumed for all CPUs : 0.000885 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:46:52] Energy consumed for all GPUs : 0.001104 kWh. Total GPU Power : 54.44724963836144 W
[codecarbon INFO @ 18:46:52] 0.002088 kWh of electricity used since the beginning.


...

Epoch 9/10, Loss: 0.0616, Accuracy: 98.13%


[codecarbon INFO @ 18:52:22] Energy consumed for RAM : 0.000534 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:52:22] Energy consumed for all CPUs : 0.004780 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:52:22] Energy consumed for all GPUs : 0.006254 kWh. Total GPU Power : 56.56654985562621 W
[codecarbon INFO @ 18:52:22] 0.011569 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:52:37] Energy consumed for RAM : 0.000554 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:52:37] Energy consumed for all CPUs : 0.004957 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:52:37] Energy consumed for all GPUs : 0.006489 kWh. Total GPU Power : 56.38514701104989 W
[codecarbon INFO @ 18:52:37] 0.012001 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:52:52] Energy consumed for RAM : 0.000574 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:52:52] Energy consumed for all CPUs : 0.005134 kWh. Total CPU Power : 42.5 W
[codeca

Epoch 10/10, Loss: 0.0162, Accuracy: 99.49%


[codecarbon INFO @ 18:53:07] Energy consumed for RAM : 0.000594 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:53:07] Energy consumed for all CPUs : 0.005311 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:53:07] Energy consumed for all GPUs : 0.006956 kWh. Total GPU Power : 55.56658419888078 W
[codecarbon INFO @ 18:53:07] 0.012861 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:53:12] Energy consumed for RAM : 0.000600 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:53:12] Energy consumed for all CPUs : 0.005368 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:53:12] Energy consumed for all GPUs : 0.007009 kWh. Total GPU Power : 39.748686086779344 W
[codecarbon INFO @ 18:53:12] 0.012978 kWh of electricity used since the beginning.


Test Accuracy: 96.72%
Total Training Time: 454.98 seconds
